In [1]:
from clickzetta.zettapark.session import Session

In [2]:
import json
# 从配置文件中读取参数
with open('security/config-uat.json', 'r') as config_file:
    config = json.load(config_file)

# 创建会话
session = Session.builder.configs(config).create()

In [3]:
data = [("111",50000),("222",60000),("333",40000)]
columns= ["EmpId","Salary"]
df = session.createDataFrame(data = data, schema = columns)
df.printSchema()
df.show()


root
 |-- `empid`: StringType() (nullable = False)
 |-- `salary`: IntegerType() (nullable = False)
------------------
|empid  |salary  |
------------------
|111    |50000   |
|222    |60000   |
|333    |40000   |
------------------



In [4]:
from clickzetta.zettapark.functions import col,lit
df2 = df.select(col("EmpId"),col("Salary"),lit("1").alias("lit_value1"))
df2.show()

-------------------------------
|empid  |salary  |lit_value1  |
-------------------------------
|111    |50000   |1           |
|222    |60000   |1           |
|333    |40000   |1           |
-------------------------------



In [5]:
from clickzetta.zettapark.functions import when
df3 = df2.withColumn("lit_value2", when((col("Salary") >= 40000) & (col("Salary") <= 50000), lit("100")).otherwise(lit("200")))
df3.show()

--------------------------------------------
|empid  |salary  |lit_value1  |lit_value2  |
--------------------------------------------
|111    |50000   |1           |100         |
|222    |60000   |1           |200         |
|333    |40000   |1           |100         |
--------------------------------------------



In [6]:
# Drop tables starting with 'zettapark_temp_table_'
try:
    tables = session.sql("SHOW TABLES LIKE 'zettapark_temp_table_%'").collect()
    
    for table in tables:
        table_name = table['table_name']
        session.sql(f"DROP TABLE IF EXISTS {table_name}").collect()
        print(f"temp Table {table_name} dropped")
except Exception as e:
    print(f"Error dropping temp tables: {e}")

# Drop tables starting with 'zettapark_temp_view_'
try:
    tables = session.sql("SHOW TABLES LIKE 'zettapark_temp_view_%'").collect()
    
    for table in tables:
        table_name = table['table_name']
        session.sql(f"DROP VIEW IF EXISTS {table_name}").collect()
        print(f"temp Table {table_name} dropped")
except Exception as e:
    print(f"Error dropping temp tables: {e}")
  

In [7]:
session.close()